# autodiff to check handcalculated derivatives

In [792]:
import numpy as np
import mygrad as mg
import random as rn

### auxiliary code to refer to later

In [793]:
x = mg.tensor(3.0)
y = mg.tensor(1.0)
z = mg.tensor(2.0)
f = x**2 + y*z
f.backward()
# partial derivatives in the direction of x, y and z in this order
print(x.grad)
print(y.grad)
print(z.grad)

6.0
2.0
1.0


In [794]:
epsilon = 1
for i in range(1,10):
    print ( ( ( (x+epsilon)**2 + y*z ) - ( x**2 + y*z ) ) / epsilon )
    epsilon /= 10

Tensor(7.)
Tensor(6.1)
Tensor(6.01)
Tensor(6.001)
Tensor(6.0001)
Tensor(6.00001)
Tensor(6.000001)
Tensor(6.00000009)
Tensor(5.99999996)


### derivatives of a vector

In [795]:
# I mean to do \partial_x(f(x,y)), where f(x,y)=[2xy, e^x * y]
x = mg.tensor(1.0)
y = mg.tensor(2.0)
f0 = 2*x*y

f0.backward()
print("f0")
print(x.grad)
print(y.grad)

f0
4.0
2.0


In [796]:
f1 = mg.exp(x)*y
f1.backward()
print("f1")
print(x.grad)
print(y.grad)

f1
5.43656365691809
2.718281828459045


# derivatives of L(x^1, ..., x^N)

In [797]:
def delta( first, second ):
    if( first is second ): return 1
    else: return 0

### centers of faces

#### by the first component

In [798]:
# the first vector
x00 = mg.tensor( 5.0 )
x01 = mg.tensor( 8.0 )
# the second vector
x10 = mg.tensor( 2.0 )
x11 = mg.tensor( 3.0 )
# the function x_sigma
xs1 = 1/2 * ( x00 + x10 )
xs2 = 1/2 * ( x01 + x11 )
print(xs1)

Tensor(3.5)


##### first component

In [799]:
xs1.backward()
print(x00.grad)
print(x10.grad)

0.5
0.5


In [800]:
eps = 1
for i in range(1,10):
    print ( ( ( 1/2 * (x00 + eps + x10) ) - ( 1/2 * ( x00 + x10 ) ) ) / eps )
    eps /= 10

Tensor(0.5)
Tensor(0.5)
Tensor(0.5)
Tensor(0.5)
Tensor(0.5)
Tensor(0.5)
Tensor(0.5)
Tensor(0.5)
Tensor(0.5)


##### second component (unsurprisingly zero)

In [801]:
x10 = mg.tensor( 2.0 )
x11 = mg.tensor( 3.0 )
xs2 = 1/2 * ( x01 + x11 )
xs2.backward()
print(x00.grad)
print(x10.grad)

None
None


##### sort of calculation by definition of a derivative

In [802]:
eps = 1
for i in range(1,10):
    print ( ( ( 1/2 * (x01 + x11) ) - ( 1/2 * ( x01 + x11 ) ) ) / eps )
    eps /= 10

Tensor(0.)
Tensor(0.)
Tensor(0.)
Tensor(0.)
Tensor(0.)
Tensor(0.)
Tensor(0.)
Tensor(0.)
Tensor(0.)


#### by the second component

In [803]:
# the first vector
x00 = mg.tensor( 4000.0 )
x01 = mg.tensor( 80.0 )
# the second vector
x10 = mg.tensor( -250.0 )
x11 = mg.tensor( 3.0 )
# the function x_sigma
xs1 = 1/2 * ( x00 + x10 )
xs2 = 1/2 * ( x01 + x11 )

##### the first component, again, this is zero

In [804]:
xs2 = 1/2 * ( x01 + x11 )
xs2.backward()
print(x00.grad)
print(x10.grad)

None
None


##### the second component

In [805]:
xs2.backward()
print(x01.grad)
print(x11.grad)

0.5
0.5


##### I hereby conclude, that the derivative of faces' centers is in deed the vector (1/2, 0) by the first component and (0, 1/2) by the second one

### face measures

In [806]:
def m(x00, x01, x10, x11):
    return mg.sqrt((x00-x10)**2+(x01-x11)**2)

###### test the function

In [807]:
print(m(12,16,6,8))

Tensor(10.)


###### define the derivatives

In [808]:
def dm1(x00, x01, x10, x11, j):
    return 1/m(x00, x01, x10, x11) * (x10-x00) * (delta(x10, j) - delta(x00, j))
def dm2(x00, x01, x10, x11, j):
    return 1/m(x00, x01, x10, x11) * (x11-x01) * (delta(x11, j) - delta(x01, j))

In [809]:
x00 = mg.tensor(50.0)
x01 = mg.tensor(10.0)
x10 = mg.tensor(40.0)
x11 = mg.tensor(30.0)
ms=m(x00, x01, x10, x11)
print(delta(x01,x01))

1


In [810]:
ms.backward()
print(x00.grad)
print(x10.grad)
print(dm1(x00,x01,x10,x11,x00))
print(dm1(x00,x01,x10,x11,x10))
print(dm2(x00,x01,x10,x11,x00))
print(dm2(x00,x01,x10,x11,x10))

0.4472135954999579
-0.4472135954999579
Tensor(0.4472136)
Tensor(-0.4472136)
Tensor(0.)
Tensor(0.)


##### robust test by generating random inputs

In [811]:
count = 0
for i in range(1000):
    x00 = mg.tensor(rn.random()*100)
    x01 = mg.tensor(rn.random()*100)
    x10 = mg.tensor(rn.random()*100)
    x11 = mg.tensor(rn.random()*100)
    ms = m(x00, x01, x10, x11)
    ms.backward()
    num = x00.grad
    difference = mg.abs(num - mg.tensor( dm1( x00, x01, x10, x11, x00 ) ) )
    if( difference > 0.000001 ):
        print(difference)
        print('for values x00:' + str(x00) + ', x01: ' + str(x01) + ', x10: ' + str(x10) + ', x11: ' + str(x11))
        count = count + 1
print( count )

0


In [812]:
count = 0
for i in range(1000):
    x00 = mg.tensor(rn.random()*100)
    x01 = mg.tensor(rn.random()*100)
    x10 = mg.tensor(rn.random()*100)
    x11 = mg.tensor(rn.random()*100)
    ms = m(x00, x01, x10, x11)
    ms.backward()
    num = x10.grad
    difference = mg.abs(num - mg.tensor( dm1( x00, x01, x10, x11, x10 ) ) )
    if( difference > 0.000001 ):
        print(difference)
        print('for values x00:' + str(x00) + ', x01: ' + str(x01) + ', x10: ' + str(x10) + ', x11: ' + str(x11))
        count = count + 1
print( count )

0


In [813]:
count = 0
for i in range(1000):
    x00 = mg.tensor(rn.random()*100)
    x01 = mg.tensor(rn.random()*100)
    x10 = mg.tensor(rn.random()*100)
    x11 = mg.tensor(rn.random()*100)
    ms = m(x00, x01, x10, x11)
    ms.backward()
    num = x01.grad
    difference = mg.abs(num - mg.tensor( dm2( x00, x01, x10, x11, x01 ) ) )
    if( difference > 0.000001 ):
        print(difference)
        print('for values x00:' + str(x00) + ', x01: ' + str(x01) + ', x10: ' + str(x10) + ', x11: ' + str(x11))
        count = count + 1
print( count )

0


In [814]:
count = 0
for i in range(1000):
    x00 = mg.tensor(rn.random()*100)
    x01 = mg.tensor(rn.random()*100)
    x10 = mg.tensor(rn.random()*100)
    x11 = mg.tensor(rn.random()*100)
    ms = m(x00, x01, x10, x11)
    ms.backward()
    num = x11.grad
    difference = mg.abs(num - mg.tensor( dm2( x00, x01, x10, x11, x11 ) ) )
    if( difference > 0.000001 ):
        print(difference)
        print('for values x00:' + str(x00) + ', x01: ' + str(x01) + ', x10: ' + str(x10) + ', x11: ' + str(x11))
        count = count + 1
print( count )

0


##### here, the derivative calculated by hand seems to be in line with the automatic derivative as well

### normals

In [815]:
def n0(x00, x01, x10, x11):
    return 1/m(x00, x01, x10, x11) * (x11 - x01)
def n1(x00, x01, x10, x11):
    return 1/m(x00, x01, x10, x11) * (-1) * (x10 - x00)
def n(x00, x01, x10, x11):
    return [n0(x00, x01, x10, x11), n1(x00, x01, x10, x11)]

###### test the function

In [816]:
x00 = mg.tensor(0.0)
x01 = mg.tensor(0.0)
x10 = mg.tensor(1.0)
x11 = mg.tensor(1.0)
print(n(x00, x01, x11, x10))

[Tensor(0.70710678), Tensor(-0.70710678)]


#### by the first component

###### define the handcalculated derivatives

In [817]:
def d0n00(x00, x01, x10, x11):
    return 1/(m(x00, x01, x10, x11)**3) * (x10 - x00) * (x11 - x01) + 1/m(x00, x01, x10, x11) * 0
def d0n10(x00, x01, x10, x11):
    return -1/(m(x00, x01, x10, x11)**3) * (x10 - x00) * (x11 - x01) + 1/m(x00, x01, x10, x11) * 0
def d0n01(x00, x01, x10, x11):
    return 1/(m(x00, x01, x10, x11)**3) * (x10 - x00) * (x00 - x10) + 1/m(x00, x01, x10, x11) * 1
def d0n11(x00, x01, x10, x11):
    return -1/(m(x00, x01, x10, x11)**3) * (x10 - x00) * (x00 - x10) + 1/m(x00, x01, x10, x11) * (-1)

###### compare autodiff with derivatives by hand

In [818]:
x00 = mg.tensor(40.0)
x01 = mg.tensor(-30.0)
x10 = mg.tensor(20.0)
x11 = mg.tensor(-10.0)

In [819]:
n0test = n0(x00, x01, x10, x11)
n0test.backward()
print(x00.grad)
print(x10.grad)
print(d0n00(x00, x01, x10, x11))
print(d0n10(x00, x01, x10, x11))

-0.017677669529663688
0.017677669529663688
Tensor(-0.01767767)
Tensor(0.01767767)


In [820]:
n1test = n1(x00, x01, x10, x11)
n1test.backward()
print(x00.grad)
print(x10.grad)
print(d0n01(x00, x01, x10, x11))
print(d0n11(x00, x01, x10, x11))

0.017677669529663688
-0.017677669529663688
Tensor(0.01767767)
Tensor(-0.01767767)


##### automated random testing

In [821]:
count = 0
for i in range(1000):
    x00 = mg.tensor(rn.random()*100)
    x01 = mg.tensor(rn.random()*100)
    x10 = mg.tensor(rn.random()*100)
    x11 = mg.tensor(rn.random()*100)
    n0test = n0(x00, x01, x10, x11)
    n0test.backward()
    num = x00.grad
    difference = mg.abs(num - mg.tensor( d0n00( x00, x01, x10, x11 ) ) )
    if( difference > 0.000001 ):
        print(difference)
        print('for values x00:' + str(x00) + ', x01: ' + str(x01) + ', x10: ' + str(x10) + ', x11: ' + str(x11))
        count = count + 1
print( count )

0


In [822]:
count = 0
for i in range(1000):
    x00 = mg.tensor(rn.random()*100)
    x01 = mg.tensor(rn.random()*100)
    x10 = mg.tensor(rn.random()*100)
    x11 = mg.tensor(rn.random()*100)
    n0test = n0(x00, x01, x10, x11)
    n0test.backward()
    num = x10.grad
    difference = mg.abs(num - mg.tensor( d0n10( x00, x01, x10, x11 ) ) )
    if( difference > 0.000001 ):
        print(difference)
        print('for values x00:' + str(x00) + ', x01: ' + str(x01) + ', x10: ' + str(x10) + ', x11: ' + str(x11))
        count = count + 1
print( count )

0


In [823]:
count = 0
for i in range(1000):
    x00 = mg.tensor(rn.random()*100)
    x01 = mg.tensor(rn.random()*100)
    x10 = mg.tensor(rn.random()*100)
    x11 = mg.tensor(rn.random()*100)
    n1test = n1(x00, x01, x10, x11)
    n1test.backward()
    num = x00.grad
    difference = mg.abs(num - mg.tensor( d0n01( x00, x01, x10, x11 ) ) )
    if( difference > 0.000001 ):
        print(difference)
        print('for values x00:' + str(x00) + ', x01: ' + str(x01) + ', x10: ' + str(x10) + ', x11: ' + str(x11))
        count = count + 1
print( count )

0


In [824]:
count = 0
for i in range(1000):
    x00 = mg.tensor(rn.random()*100)
    x01 = mg.tensor(rn.random()*100)
    x10 = mg.tensor(rn.random()*100)
    x11 = mg.tensor(rn.random()*100)
    n1test = n1(x00, x01, x10, x11)
    n1test.backward()
    num = x10.grad
    difference = mg.abs(num - mg.tensor( d0n11( x00, x01, x10, x11 ) ) )
    if( difference > 0.000001 ):
        print(difference)
        print('for values x00:' + str(x00) + ', x01: ' + str(x01) + ', x10: ' + str(x10) + ', x11: ' + str(x11))
        count = count + 1
print( count )

0


#### by the second component

In [825]:
x00 = mg.tensor(40.0)
x01 = mg.tensor(-30.0)
x10 = mg.tensor(20.0)
x11 = mg.tensor(-10.0)

##### define the handcalculated derivatives

In [830]:
def d0n00(x00, x01, x10, x11):
    return 1/(m(x00, x01, x10, x11)**3) * (x10 - x00) * (x11 - x01) + 1/m(x00, x01, x10, x11) * 0
def d0n10(x00, x01, x10, x11):
    return -1/(m(x00, x01, x10, x11)**3) * (x10 - x00) * (x11 - x01) + 1/m(x00, x01, x10, x11) * 0
def d0n01(x00, x01, x10, x11):
    return 1/(m(x00, x01, x10, x11)**3) * (x10 - x00) * (x00 - x10) + 1/m(x00, x01, x10, x11) * 1
def d0n11(x00, x01, x10, x11):
    return -1/(m(x00, x01, x10, x11)**3) * (x10 - x00) * (x00 - x10) + 1/m(x00, x01, x10, x11) * (-1)

In [826]:
def d1n00(x00, x01, x10, x11):
    return 1/(m(x00, x01, x10, x11)**3) * (x11 - x01) * (x11 - x01) + 1/m(x00, x01, x10, x11) * (-1)
def d1n10(x00, x01, x10, x11):
    return -1/(m(x00, x01, x10, x11)**3) * (x11 - x01) * (x11 - x01) + 1/m(x00, x01, x10, x11) * 1
def d1n01(x00, x01, x10, x11):
    return 1/(m(x00, x01, x10, x11)**3) * (x11 - x01) * (x00 - x10) + 1/m(x00, x01, x10, x11) * 0
def d1n11(x00, x01, x10, x11):
    return -1/(m(x00, x01, x10, x11)**3) * (x11 - x01) * (x00 - x10) + 1/m(x00, x01, x10, x11) * 0

##### test

In [831]:
n0test = n0(x00, x01, x10, x11)
n0test.backward()
print(x01.grad)
print(x11.grad)
print(d1n00(x00, x01, x10, x11))
print(d1n10(x00, x01, x10, x11))

-3.395354806539838e-06
3.395354806539838e-06
Tensor(-3.39535481e-06)
Tensor(3.39535481e-06)


In [832]:
n1test = n1(x00, x01, x10, x11)
n1test.backward()
print(x01.grad)
print(x11.grad)
print(d1n01(x00, x01, x10, x11))
print(d1n11(x00, x01, x10, x11))

0.00018654725605780093
-0.00018654725605780093
Tensor(0.00018655)
Tensor(-0.00018655)


In [834]:
count = 0
for i in range(1000):
    x00 = mg.tensor(rn.random()*100)
    x01 = mg.tensor(rn.random()*100)
    x10 = mg.tensor(rn.random()*100)
    x11 = mg.tensor(rn.random()*100)
    n0test = n0(x00, x01, x10, x11)
    n0test.backward()
    num = x01.grad
    difference = mg.abs(num - mg.tensor( d1n00( x00, x01, x10, x11 ) ) )
    if( difference > 0.000001 ):
        print(difference)
        print('for values x00:' + str(x00) + ', x01: ' + str(x01) + ', x10: ' + str(x10) + ', x11: ' + str(x11))
        count = count + 1
print( count )

0


In [836]:
count = 0
for i in range(1000):
    x00 = mg.tensor(rn.random()*100)
    x01 = mg.tensor(rn.random()*100)
    x10 = mg.tensor(rn.random()*100)
    x11 = mg.tensor(rn.random()*100)
    n0test = n0(x00, x01, x10, x11)
    n0test.backward()
    num = x11.grad
    difference = mg.abs(num - mg.tensor( d1n10( x00, x01, x10, x11 ) ) )
    if( difference > 0.000001 ):
        print(difference)
        print('for values x00:' + str(x00) + ', x01: ' + str(x01) + ', x10: ' + str(x10) + ', x11: ' + str(x11))
        count = count + 1
print( count )

0


In [837]:
count = 0
for i in range(1000):
    x00 = mg.tensor(rn.random()*100)
    x01 = mg.tensor(rn.random()*100)
    x10 = mg.tensor(rn.random()*100)
    x11 = mg.tensor(rn.random()*100)
    n1test = n1(x00, x01, x10, x11)
    n1test.backward()
    num = x01.grad
    difference = mg.abs(num - mg.tensor( d1n01( x00, x01, x10, x11 ) ) )
    if( difference > 0.000001 ):
        print(difference)
        print('for values x00:' + str(x00) + ', x01: ' + str(x01) + ', x10: ' + str(x10) + ', x11: ' + str(x11))
        count = count + 1
print( count )

0


In [838]:
count = 0
for i in range(1000):
    x00 = mg.tensor(rn.random()*100)
    x01 = mg.tensor(rn.random()*100)
    x10 = mg.tensor(rn.random()*100)
    x11 = mg.tensor(rn.random()*100)
    n1test = n1(x00, x01, x10, x11)
    n1test.backward()
    num = x11.grad
    difference = mg.abs(num - mg.tensor( d1n11( x00, x01, x10, x11 ) ) )
    if( difference > 0.000001 ):
        print(difference)
        print('for values x00:' + str(x00) + ', x01: ' + str(x01) + ', x10: ' + str(x10) + ', x11: ' + str(x11))
        count = count + 1
print( count )

0


### cell measures

In [599]:
def S0(x00, x01, x10, x11, x20, x21):
    return x20*x11 - x00*x11 - x20*x01 - x10*x21 + x10*x01 + x00*x21
def S(x00, x01, x10, x11, x20, x21):
    if(S0(x00, x01, x10, x11, x20, x21) > 0): return 1
    elif(S0(x00, x01, x10, x11, x20, x21) < 0): return -1
    else: return 0

In [600]:
def m2(x00, x01, x10, x11, x20, x21):
    return mg.abs(S0(x00, x01, x10, x11, x20, x21)) / 2

###### test the function

In [610]:
x00 = mg.tensor(10.0)
x01 = mg.tensor(20.0)
x10 = mg.tensor(90.0)
x11 = mg.tensor(30.0)
x20 = mg.tensor(60.0)
x21 = mg.tensor(40.0)
print(m2(x00, x01, x10, x11, x20, x21))
m2test = m2(x00, x01, x10, x11, x20, x21)
m2test.backward()
print(x00.grad)
print(x01.grad)
print(x10.grad)
print(x11.grad)
print(x20.grad)
print(x21.grad)

Tensor(550.)
-5.0
-15.0
10.0
-25.0
-5.0
40.0


#### by the first component

In [611]:
def d00m2(x00, x01, x10, x11, x20, x21):
    return -1/(2*m2(x00,x01,x10,x11,x20,x21)**2)*S(x00,x01,x10,x11,x20,x21)*(-x11+x21)
def d10m2(x00, x01, x10, x11, x20, x21):
    return -1/(2*m2(x00,x01,x10,x11,x20,x21)**2)*S(x00,x01,x10,x11,x20,x21)*(-x21+x01)
def d20m2(x00, x01, x10, x11, x20, x21):
    return -1/(2*m2(x00,x01,x10,x11,x20,x21)**2)*S(x00,x01,x10,x11,x20,x21)*(-x01+x11)

In [612]:
m2test_inverted = 1/m2(x00, x01, x10, x11, x20, x21)
m2test_inverted.backward()
print(x00.grad)
print(x10.grad)
print(x20.grad)
print(d00m2(x00, x01, x10, x11, x20, x21))
print(d10m2(x00, x01, x10, x11, x20, x21))
print(d20m2(x00, x01, x10, x11, x20, x21))

1.6528925619834715e-05
-3.305785123966942e-05
1.6528925619834708e-05
Tensor(1.65289256e-05)
Tensor(-3.30578512e-05)
Tensor(1.65289256e-05)


In [732]:
count = 0
for i in range(1000):
    x00 = mg.tensor(rn.random()*100)
    x01 = mg.tensor(rn.random()*100)
    x10 = mg.tensor(rn.random()*100)
    x11 = mg.tensor(rn.random()*100)
    x20 = mg.tensor(rn.random()*100)
    x21 = mg.tensor(rn.random()*100)
    m2test_inverted = 1/m2(x00, x01, x10, x11, x20, x21)
    m2test_inverted.backward()
    num = x00.grad
    difference = mg.abs(num - mg.tensor( d00m2( x00, x01, x10, x11, x20, x21 ) ) )
    if( difference > 0.000001 ):
        print(difference)
        print('for values x00:' + str(x00) + ', x01: ' + str(x01) +
              ', x10: ' + str(x10) + ', x11: ' + str(x11) +
              ', x20 ' + str(x20) + ', x21:' + str(x21))
        count = count + 1
print( count )

count = 0
for i in range(1000):
    x00 = mg.tensor(rn.random()*100)
    x01 = mg.tensor(rn.random()*100)
    x10 = mg.tensor(rn.random()*100)
    x11 = mg.tensor(rn.random()*100)
    x20 = mg.tensor(rn.random()*100)
    x21 = mg.tensor(rn.random()*100)
    m2test_inverted = 1/m2(x00, x01, x10, x11, x20, x21)
    m2test_inverted.backward()
    num = x10.grad
    difference = mg.abs(num - mg.tensor( d10m2( x00, x01, x10, x11, x20, x21 ) ) )
    if( difference > 0.000001 ):
        print(difference)
        print('for values x00:' + str(x00) + ', x01: ' + str(x01) +
              ', x10: ' + str(x10) + ', x11: ' + str(x11) +
              ', x20 ' + str(x20) + ', x21:' + str(x21))        
        count = count + 1
print( count )

count = 0
for i in range(1000):
    x00 = mg.tensor(rn.random()*100)
    x01 = mg.tensor(rn.random()*100)
    x10 = mg.tensor(rn.random()*100)
    x11 = mg.tensor(rn.random()*100)
    x20 = mg.tensor(rn.random()*100)
    x21 = mg.tensor(rn.random()*100)
    m2test_inverted = 1/m2(x00, x01, x10, x11, x20, x21)
    m2test_inverted.backward()
    num = x20.grad
    difference = mg.abs(num - mg.tensor( d20m2( x00, x01, x10, x11, x20, x21 ) ) )
    if( difference > 0.000001 ):
        print(difference)
        print('for values x00:' + str(x00) + ', x01: ' + str(x01) +
              ', x10: ' + str(x10) + ', x11: ' + str(x11) +
              ', x20 ' + str(x20) + ', x21:' + str(x21))        
        count = count + 1
print( count )

0
0
0


#### by the second component

In [613]:
def d01m2(x00, xo01, x10, x11, x20, x21):
    return -1/(2*m2(x00,x01,x10,x11,x20,x21)**2)*S(x00,x01,x10,x11,x20,x21)*(-x20+x10)
def d11m2(x00, xo01, x10, x11, x20, x21):
    return -1/(2*m2(x00,x01,x10,x11,x20,x21)**2)*S(x00,x01,x10,x11,x20,x21)*(-x00+x20)
def d21m2(x00, xo01, x10, x11, x20, x21):
    return -1/(2*m2(x00,x01,x10,x11,x20,x21)**2)*S(x00,x01,x10,x11,x20,x21)*(-x10+x00)

In [609]:
m2test_inverted = 1/m2(x00, x01, x10, x11, x20, x21)
m2test_inverted.backward()
print(x01.grad)
print(x11.grad)
print(x21.grad)
print(d01m2(x00, x01, x10, x11, x20, x21))
print(d11m2(x00, x01, x10, x11, x20, x21))
print(d21m2(x00, x01, x10, x11, x20, x21))

-1.7777777777777777
2.0
-0.22222222222222232
Tensor(-1.77777778)
Tensor(2.)
Tensor(-0.22222222)


In [733]:
count = 0
for i in range(1000):
    x00 = mg.tensor(rn.random()*100)
    x01 = mg.tensor(rn.random()*100)
    x10 = mg.tensor(rn.random()*100)
    x11 = mg.tensor(rn.random()*100)
    x20 = mg.tensor(rn.random()*100)
    x21 = mg.tensor(rn.random()*100)
    m2test_inverted = 1/m2(x00, x01, x10, x11, x20, x21)
    m2test_inverted.backward()
    num = x01.grad
    difference = mg.abs(num - mg.tensor( d01m2( x00, x01, x10, x11, x20, x21 ) ) )
    if( difference > 0.000001 ):
        print(difference)
        print('for values x00:' + str(x00) + ', x01: ' + str(x01) +
              ', x10: ' + str(x10) + ', x11: ' + str(x11) +
              ', x20 ' + str(x20) + ', x21:' + str(x21))
        count = count + 1
print( count )

count = 0
for i in range(1000):
    x00 = mg.tensor(rn.random()*100)
    x01 = mg.tensor(rn.random()*100)
    x10 = mg.tensor(rn.random()*100)
    x11 = mg.tensor(rn.random()*100)
    x20 = mg.tensor(rn.random()*100)
    x21 = mg.tensor(rn.random()*100)
    m2test_inverted = 1/m2(x00, x01, x10, x11, x20, x21)
    m2test_inverted.backward()
    num = x11.grad
    difference = mg.abs(num - mg.tensor( d11m2( x00, x01, x10, x11, x20, x21 ) ) )
    if( difference > 0.000001 ):
        print(difference)
        print('for values x00:' + str(x00) + ', x01: ' + str(x01) +
              ', x10: ' + str(x10) + ', x11: ' + str(x11) +
              ', x20 ' + str(x20) + ', x21:' + str(x21))        
        count = count + 1
print( count )

count = 0
for i in range(1000):
    x00 = mg.tensor(rn.random()*100)
    x01 = mg.tensor(rn.random()*100)
    x10 = mg.tensor(rn.random()*100)
    x11 = mg.tensor(rn.random()*100)
    x20 = mg.tensor(rn.random()*100)
    x21 = mg.tensor(rn.random()*100)
    m2test_inverted = 1/m2(x00, x01, x10, x11, x20, x21)
    m2test_inverted.backward()
    num = x21.grad
    difference = mg.abs(num - mg.tensor( d21m2( x00, x01, x10, x11, x20, x21 ) ) )
    if( difference > 0.000001 ):
        print(difference)
        print('for values x00:' + str(x00) + ', x01: ' + str(x01) +
              ', x10: ' + str(x10) + ', x11: ' + str(x11) +
              ', x20 ' + str(x20) + ', x21:' + str(x21))        
        count = count + 1
print( count )

0
0
0
